# Section 2

## Scenario 1 
---
Download the passing rate of annual vehicle inspection records from 
Some forum posters have complained that the value of their HDB flats suffer because they are near 
expressways, which are very noisy. Others say expressway proximity is good, due to the unblocked 
view (at least for higher floors). <br/>
 <br/>
The Housing and Development Board has tasked your team to analyse whether there is merit to 
either view, based on transaction prices for resale HDB flats in recent years.

In [1]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#!pip install geopandas
import geopandas as gpd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [2]:
#Importing housing datasets
df1 = pd.read_csv("resale-flat-prices-based-on-approval-date-1990-1999.csv")
df2 = pd.read_csv("resale-flat-prices-based-on-approval-date-2000-feb-2012.csv")
df3 = pd.read_csv("resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv")
df4 = pd.read_csv("resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")
df5 = pd.read_csv("resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv")
resale_df = pd.concat([df1,df2,df3,df4,df5])

In [7]:
#Importing map line dataset
data = gpd.read_file('national-map-line-geojson.geojson')
df_list = []
def get_df(df_list, value):
    temp = pd.read_html(str(value))
    temp_df = pd.DataFrame(temp[0]).set_index('Attributes').T.reset_index(drop=True)
    df_list.append(temp_df)

#Get data from Description column and then concat all data into df
data.Description.apply(lambda x: get_df(df_list,x))
map_df = pd.concat(df_list, axis=0).reset_index(drop=True)

Attributes,NAME,FOLDERPATH,SYMBOLID,INC_CRC,FMEL_UPD_D
0,CENTRAL EXPRESSWAY,Layers/Expressway_Sliproad,2,0C08DFFA475DDCCD,20191008154530
1,CENTRAL EXPRESSWAY,Layers/Expressway_Sliproad,2,48A90A617CC124B8,20191008154530
2,CENTRAL EXPRESSWAY,Layers/Expressway_Sliproad,2,051AA478B6209021,20191008154530
3,CENTRAL EXPRESSWAY,Layers/Expressway_Sliproad,2,1C51FD53E1662A6B,20191008154530
4,CENTRAL EXPRESSWAY,Layers/Expressway_Sliproad,2,44D0FFDF1EF47027,20191008154530
...,...,...,...,...,...
5130,40,Layers/Contour_250K,4,8335AEC2469FDB5A,20191008154531
5131,20,Layers/Contour_250K,4,7341E9E668DAA485,20191008154531
5132,20,Layers/Contour_250K,4,80474D20EDA042D7,20191008154531
5133,20,Layers/Contour_250K,4,CEF6E059889EB780,20191008154531


In [8]:
#Preliminary check of each column's data type and values
for col in resale_df.columns:
    print(resale_df[col].dtype, col, resale_df[col].unique()[:10])

object month ['1990-01' '1990-02' '1990-03' '1990-04' '1990-05' '1990-06' '1990-07'
 '1990-08' '1990-09' '1990-10']
object town ['ANG MO KIO' 'BEDOK' 'BISHAN' 'BUKIT BATOK' 'BUKIT MERAH' 'BUKIT TIMAH'
 'CENTRAL AREA' 'CHOA CHU KANG' 'CLEMENTI' 'GEYLANG']
object flat_type ['1 ROOM' '3 ROOM' '4 ROOM' '5 ROOM' '2 ROOM' 'EXECUTIVE'
 'MULTI GENERATION' 'MULTI-GENERATION']
object block ['309' '216' '211' '202' '235' '232' '308' '220' '219' '247']
object street_name ['ANG MO KIO AVE 1' 'ANG MO KIO AVE 3' 'ANG MO KIO AVE 4'
 'ANG MO KIO AVE 10' 'ANG MO KIO AVE 5' 'ANG MO KIO AVE 8'
 'ANG MO KIO AVE 6' 'ANG MO KIO AVE 9' 'ANG MO KIO AVE 2'
 'BEDOK RESERVOIR RD']
object storey_range ['10 TO 12' '04 TO 06' '07 TO 09' '01 TO 03' '13 TO 15' '19 TO 21'
 '16 TO 18' '25 TO 27' '22 TO 24' '28 TO 30']
float64 floor_area_sqm [31. 73. 67. 82. 74. 88. 89. 83. 68. 75.]
object flat_model ['IMPROVED' 'NEW GENERATION' 'MODEL A' 'STANDARD' 'SIMPLIFIED'
 'MODEL A-MAISONETTE' 'APARTMENT' 'MAISONETTE' 'TERRACE' '2

In [9]:
#Preliminary check of each column's data type and values
for col in map_df.columns:
    print(map_df[col].dtype, col, map_df[col].unique()[:10])

object NAME ['CENTRAL EXPRESSWAY' 'CHANGI COAST ROAD' 'CHANGI NORTH CRESCENT'
 'CHANGI SOUTH AVENUE 3' 'CHIN SWEE ROAD' 'CHOA CHU KANG DRIVE'
 'LORNIE HIGHWAY' 'TAMPINES AVENUE 10' 'XILIN AVENUE' 'CHOA CHU KANG WAY']
object FOLDERPATH ['Layers/Expressway_Sliproad' 'Layers/Expressway'
 'Layers/International_bdy' 'Layers/Major_Road' 'Layers/Contour_250K']
object SYMBOLID ['2' '1' '0' '3' '4']
object INC_CRC ['0C08DFFA475DDCCD' '48A90A617CC124B8' '051AA478B6209021'
 '1C51FD53E1662A6B' '44D0FFDF1EF47027' 'FB1312F3B35BE59C'
 'FC79D834475C81E6' '124BF3A9A7B00559' '4D32A9DCA4D3A3FD'
 '17AFD7EBA474B2CE']
object FMEL_UPD_D ['20191008154530' '20191008154531']


In [ ]:
map_df[map_df.NAME.str.contains("EXPRESSWAY")]